In [1]:
!pip install pillow
!pip install SSIM_PIL

In [2]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from SSIM_PIL import compare_ssim
import numpy as np
import os

In [3]:
def text2png(text, path, color = "#000", bgcolor = "#FFF", font_path = None, font_size = 8, size=(14,17)):
    font = ImageFont.load_default() if font_path == None else ImageFont.truetype(font_path, font_size)
    
    img = Image.new("RGBA", size, bgcolor)
    draw = ImageDraw.Draw(img)
    draw.text( (0, 0), text, color, font=font)

    img.save(path)

In [4]:
def get_ssim_matrix(similarity_power, font_path="ssim/DroidSansMono.ttf", font_size=9, image_path='ssim/images'):
    # get image dimensions from largest character in font
    font = ImageFont.truetype(font_path, font_size)
    max_size = (0, 0)
    for i in range(32, 127): # ascii printable characters
        text_size = font.getsize(chr(i))
        max_size = (max(max_size[0], text_size[0]), max(max_size[1], text_size[1]))
    # generate character images
    for i in range(32, 127):
        text = chr(i)
        out = f'{image_path}/{str(i)}.png'
        text2png(text, out, size=max_size, font_path=font_path, font_size=font_size)
    # generate matrix for pairwise SSIM (higher similarity_power values will accentuate differences more)
    ssim_matrix = np.zeros((95, 95))
    for i in range(32, 127):
        for j in range(i + 1, 127):
            ssim_matrix[i - 32, j - 32] = compare_ssim(
                Image.open(f'{image_path}/{str(i)}.png'),
                Image.open(f'{image_path}/{str(j)}.png'),
                GPU=False
            )
    ssim_matrix += ssim_matrix.T + np.eye(95)
    return ssim_matrix ** similarity_power

In [5]:
os.makedirs('ssim/images', exist_ok=True)

In [6]:
ssim_matrix = get_ssim_matrix(8)

In [7]:
char_list = np.array([chr(i) for i in range(32, 127)])
char_list

array([' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',',
       '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F',
       'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
       'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
       '{', '|', '}', '~'], dtype='<U1')

In [8]:
def ocr_levenshtein(s, t, return_type='ratio', ssim_matrix=None, sub_weight=2, ssim_sub_only=False):
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows, cols))

    distance[:, 0] = np.arange(rows)
    distance[0, :] = np.arange(cols)

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            del_cost = 1
            ins_cost = 1
            sub_cost = 1 * sub_weight
            
            if s[row-1] == t[col-1]:
                sub_cost = 0
            elif ssim_matrix is not None:
                ssim_row = ord(s[row-1]) - 32
                ssim_col = ord(t[col-1]) - 32
                if (0 <= ssim_row <= ssim_matrix.shape[0] and
                    0 <= ssim_col <= ssim_matrix.shape[1]):
                    sub_cost = (1 - ssim_matrix[ssim_row, ssim_col]) * sub_weight
                    if not ssim_sub_only:
                        del_cost = 1 - ssim_matrix[ssim_row, 0]
                        ins_cost = 1 - ssim_matrix[0, ssim_col]
            
            distance[row, col] = min(distance[row-1, col] + del_cost, # Cost of deletions
                                 distance[row, col-1] + ins_cost,     # Cost of insertions
                                 distance[row-1, col-1] + sub_cost)   # Cost of substitutions
    if return_type == 'ratio':
        return ((len(s) + len(t)) - distance[-1, -1]) / (len(s) + len(t))
    elif return_type == 'distance':
        return distance[-1, -1]
    elif return_type == 'matrix':
        return distance[1:, 1:]

In [9]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
print(int(ocr_levenshtein("Hello", "He11O", ssim_matrix=ssim_matrix) * 100))
print(fuzz.ratio("Hello", "He11O"))

50
40


In [11]:
print(int(ocr_levenshtein("foo", "bar", ssim_matrix=ssim_matrix) * 100))
print(fuzz.ratio("foo", "bar"))

17
0


In [12]:
print(int(ocr_levenshtein("Apple", "AppIe", ssim_matrix=ssim_matrix) * 100))
print(fuzz.ratio("Apple", "AppIe"))

92
80


In [13]:
print(int(ocr_levenshtein("Apple", "Orange", ssim_matrix=ssim_matrix) * 100))
print(fuzz.ratio("Apple", "Orange"))

39
18
